In [55]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [71]:
import requests
import pandas as pd
import json
import time
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta

#Documentação: https://developers.rdstation.com/reference/get_platform-segmentations-id-contacts

In [72]:
load_dotenv()  # Isso carrega as variáveis do arquivo .env

CLIENT_ID = os.getenv("CLIENT_ID_RD")
CLIENT_SECRET = os.getenv("CLIENT_SECRET_RD")
REFRESH_TOKEN= os.getenv("REFRESH_TOKEN_RD")

if not CLIENT_ID or not CLIENT_SECRET:
    raise EnvironmentError("CLIENT_ID_RD ou CLIENT_SECRET_RD não foram encontrados nas variáveis de ambiente.")

In [73]:
# ---- BUSCA O ACCESS_TOKEN ---- #
url = "https://api.rd.services/auth/token"

payload = {
    "client_id": f'{CLIENT_ID}',
    "client_secret": f'{CLIENT_SECRET}',
    "refresh_token": f'{REFRESH_TOKEN}'
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

dados = response.json()

access_token = dados['access_token']

In [77]:
# ---- CARREGA TODOS OS LEADS DO RD STATION ----#
HEADERS_RD = {
    "accept": "application/json",
    "authorization": f"Bearer {access_token}"
}

base_url = "https://api.rd.services/platform/segmentations/13513770/contacts"
#base_url = "https://api.rd.services/platform/segmentations/13513770/contacts?order=last_conversion_date:asc"

page = 1
page_size = 125

todos_os_leads = []

while True:
    url = f"{base_url}?page={page}&page_size={page_size}"
    response = requests.get(url, headers=HEADERS_RD)

    if response.status_code != 200:
        print(f"Erro na página {page}: {response.status_code} - {response.text}")
        break

    resultado = response.json()
    contatos = resultado.get("contacts", [])

    if not contatos:
        print(f"Fim da paginação. Nenhum contato na página {page}.")
        break

    df_temp = pd.json_normalize(contatos)
    todos_os_leads.append(df_temp)

    print(f"Página {page} carregada com {len(df_temp)} contatos.")
    page += 1

# Junta todos os DataFrames em um só
df_final = pd.concat(todos_os_leads, ignore_index=True)

print(f"\nTotal de contatos obtidos: {len(df_final)}")

Página 1 carregada com 125 contatos.
Página 2 carregada com 125 contatos.
Página 3 carregada com 125 contatos.
Página 4 carregada com 125 contatos.
Página 5 carregada com 125 contatos.
Página 6 carregada com 125 contatos.
Página 7 carregada com 125 contatos.
Página 8 carregada com 125 contatos.
Página 9 carregada com 125 contatos.
Página 10 carregada com 125 contatos.
Página 11 carregada com 125 contatos.
Página 12 carregada com 125 contatos.
Página 13 carregada com 125 contatos.
Página 14 carregada com 125 contatos.
Página 15 carregada com 125 contatos.
Página 16 carregada com 125 contatos.
Página 17 carregada com 125 contatos.
Página 18 carregada com 117 contatos.
Fim da paginação. Nenhum contato na página 19.

Total de contatos obtidos: 2242


In [78]:
# --- FILTRAR EXATAMENTE OS ÚLTIMOS 7 DIAS ---
# Edite o parâmetro de data aqui caso queira um período maior
data_filtro = (datetime.utcnow() - timedelta(days=7)).strftime("%Y-%m-%d")

df_filtrado = df_final[df_final['last_conversion_date'] >= data_filtro]

2025-04-19
                                     uuid                            name  \
0    fdedc1bb-4c78-4482-98bc-224d47507f2d  Délis Edwirges Silva Oliveira    
1    57d1f109-654b-4dd0-8832-305d49888bad       Beatriz Lucena dos Santos   
2    f169d704-701f-4bc0-b128-65ad7ce07e70                Beatriz Barssoti   
3    3fe5eb7c-e17f-4a91-ae12-3984ee444517                 Tatiane s silva   
4    98633964-8a4a-43c9-9118-7b86dc4ce6ab                  Amanda Tintino   
..                                    ...                             ...   
248  f19ba91a-4fe8-48a4-8759-fbe36892ac89              Sthefani Francisca   
249  65e908ec-1752-43d7-a514-f536a49855c1          Daniele Leal de Araújo   
250  ee702843-eb7c-4e69-96ec-5ce2c65f1ae8             Ariely Souza Capile   
251  2a8e29d7-8c88-4bb0-9a51-a8f25be1f95e                 Gabrielly Pavan   
252  4d64988c-02de-4f07-b564-4030cdc28c17          Brenda da Silva Garcia   

                            email           last_conversion_date

In [79]:
# ===== NOTION CONFIG =====

NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_DATABASE_ID = '1bfe8390748f80389530f5f13fa7c9c6'
HEADERS_NOTION = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
}

In [80]:
def get_contact_conversions(contact_uuid):
    url = f"https://api.rd.services/platform/contacts/{contact_uuid}/events?event_type=CONVERSION"
    response = requests.get(url, headers=HEADERS_RD)
    
    data = response.json()
    event_identifiers = []
    for event in data:
        event_identifiers.append(event.get("event_identifier"))
    return event_identifiers


In [81]:
def buscar_lead_por_email(email):
    url = "https://api.notion.com/v1/databases/{}/query".format(NOTION_DATABASE_ID)
    payload = {
        "filter": {
            "property": "Email",
            "rich_text": {
                "equals": email
            }
        }
    }

    response = requests.post(url, headers=HEADERS_NOTION, json=payload)
    data = response.json()

    results = data.get("results", [])
    if results:
        return results[0]["id"]  # Retorna o ID da página encontrada
    return None

def criar_ou_atualizar_lead(name, email, linkedin, nick_discord, landing_pages):
    # Monta o campo multi-select
    multi_select = [{"name": lp} for lp in sorted(set(landing_pages))]

    page_id = buscar_lead_por_email(email)

    payload = {
        "properties": {
            "Name": {"title": [{"text": {"content": name}}]},
            "Email": {"email": email},
            "Linkedin": {"rich_text": [{"text": {"content": linkedin}}]},
            "Nick Discord": {"rich_text": [{ "text": {"content": nick_discord}}]},
            "landing-page": {"multi_select": multi_select}
        }
    }

    if page_id:
        # Atualiza registro existente
        url = f"https://api.notion.com/v1/pages/{page_id}"
        response = requests.patch(url, headers=HEADERS_NOTION, json=payload)
        print(f"Lead atualizado: {email}")
    else:
        # Cria novo registro
        payload["parent"] = {"database_id": NOTION_DATABASE_ID}
        url = "https://api.notion.com/v1/pages"
        response = requests.post(url, headers=HEADERS_NOTION, json=payload)
        print(f"Lead criado: {email}")

    if response.status_code not in [200, 201]:
        print("Status Code:", response.status_code)
        print(f"Erro ao salvar lead {email}: {response.text}")

In [83]:
# ===== GRAVAR NO NOTION =====


for email in df_filtrado['email']:
    
    url = f'https://api.rd.services/platform/contacts/email:{email}'
    response = requests.get(url, headers=HEADERS_RD)
    data = response.json()

    uuid = data.get('uuid')

    landing_page = get_contact_conversions(uuid)

    email = data.get('email')

    name = data.get('name')

    if data.get('linkedin')==None:
        linkedin = ""
    else:
        linkedin = data.get('linkedin')
    
    if data.get('cf_nick_discord')==None:
        nick_discord=""
    else: 
        nick_discord = data.get('cf_nick_discord')

    criar_ou_atualizar_lead(name, email, linkedin, nick_discord, landing_page)
    time.sleep(1)


Lead atualizado: delis.edwirges@hotmail.com
Lead atualizado: beatrizlucena2007@gmail.com
Lead atualizado: beatriz.barssoti07@gmail.com
Lead atualizado: tati.silva2509@gmail.com
Lead atualizado: amandatintino@gmail.com
Lead atualizado: ge_cris@yahoo.com.br
Lead atualizado: bl648559@gmail.com
Lead atualizado: kvicente.santos32@gmail.com
Lead atualizado: danydayane2@hotmail.com
Lead atualizado: thaisouza9999@gmail.com
Lead atualizado: yasmimferreiramarcolino@gmail.com
Lead atualizado: mayarasdavid@yahoo.com.br
Lead atualizado: ohashithais@gmail.com
Lead atualizado: larissaaguiarvagas@gmail.com
Lead atualizado: assis.leticiasa@gmail.com
Lead atualizado: belinhasisa9@gmail.com
Lead atualizado: mariana.mello1@fatec.sp.gov.br
Lead atualizado: anamorais192@gmail.com
Lead atualizado: kamila.cordier@fenixcloud.com.br
Lead atualizado: bialmeidamello@gmail.com
Lead atualizado: viviane86alves@gmail.com
Lead atualizado: izabelabonzanini@gmail.com
Lead atualizado: naiara_freire02@hotmail.com
Lead atu